## Project - Bank Churn prediction

 
### In this project, we aim to predict the churn for a bank, i.e, given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?

### Objective:
Given a Bank customer, build a neural network based classifier that can determine whether they will leave or not in the next 6 months. 
### Context:
Businesses like banks which provide service have to worry about problem of 'Churn' i.e. customers leaving and joining another service provider. It is important to understand which aspects of the service influence a customer's decision in this regard. Management can concentrate efforts on improvement of service, keeping in mind these priorities.

### Data Description:
The case study is from an open-source dataset from Kaggle.The dataset contains 10,000 sample points with 14 distinct features such as CustomerId, CreditScore, Geography, Gender, Age, Tenure, Balance etc.
Link to the Kaggle project site:https://www.kaggle.com/barelydedicated/bank-customer-churn-modelingPoints 

### Distribution:
The points distribution for this case is as follows:
- 1. Read the dataset
- 2. Drop the columns which are unique for all users like IDs (5 points)
- 3. Distinguish the feature and target set (5 points)
- 4. Divide the data set into trainingand test sets (5 points)
- 5. Normalize the train and test data(10points)
- 6. Initialize & build the model. Identify the points of improvement and implement the same the same.(20)
- 7. Predict the results using 0.5 as a threshold (10points)8. Print the Accuracy score and confusion matrix (5 points)

In [ ]:
!pip install tensorflow==2.0

     |████████████████████████████████| 86.3MB 67kB/s 
     |████████████████████████████████| 3.8MB 36.9MB/s 
     |████████████████████████████████| 450kB 46.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=01bc436098ac3a0be52e90fa06704ec65f7af2571bcdff549cadf534a89d2b05
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.2
    Uninstalling tensorflow-1.15.2:
      Successfully uninstalled tensorflow-1.15.2


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Read the dataset

In [ ]:
ds = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Project - Bank Churn Prediction/bank_1.csv")

In [ ]:
ds.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [ ]:
#RowNumber #CustomerId and #Surname are unique hence dropping it
ds = ds.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
ds['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [ ]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


#### Distinguish the feature and target set

In [ ]:
X = ds.iloc[:,0:10].values # Credit Score through Estimated Salary
y = ds.iloc[:,10].values # Exited

In [ ]:
# Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany
# This will convert those strings into scalar values for analysis
print(X[:8,1], '... will now become: ')

label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


In [ ]:
# We will do the same thing for gender. this will be binary in this dataset
print(X[:6,2], '... will now become: ')

label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


In [ ]:
X.shape

(10000, 10)

In [ ]:
# The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). 
# Therefore, one way to get rid of that problem is to split the countries into respective dimensions.
# Gender does not need this as it is binary

# Converting the string features into their own dimensions. Gender doesn't matter here because its binary
#countryhotencoder = OneHotEncoder(categories = [1]) # 1 is the country column
countryhotencoder = ColumnTransformer([("countries", OneHotEncoder(), [1])], remainder="passthrough")
X = countryhotencoder.fit_transform(X)

In [ ]:
X.shape

(10000, 12)

In [ ]:
X[0]

array([1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
# A 0 on two countries means that the country has to be the one variable which wasn't included 
# This will save us from the problem of using too many dimensions
X = X[:,1:] # Got rid of Spain as a dimension.

In [ ]:
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

#### Divide the data set into Train and test sets

In [ ]:
# Splitting the dataset into the Training and Testing set.

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.size

88000

In [ ]:
X_test.size

22000

#### Normalize the train and test data

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Initialize & build the model

In [ ]:
# Initializing the ANN
classifier = Sequential()

In [ ]:
# The amount of nodes (dimensions) in hidden layer should be the average of input and output layers, in this case 6.
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [ ]:
#Add 1st hidden layer
classifier.add(Dense(6, activation='relu', kernel_initializer='uniform'))

In [ ]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [ ]:
# Create optimizer with default learning rate
# Compile the model
classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=10,
          batch_size=32)

Epoch 1/10
250/250 [==============================] - 3s 6ms/step - loss: 0.2277 - accuracy: 0.7960 - val_loss: 0.2094 - val_accuracy: 0.7975
Epoch 2/10
250/250 [==============================] - 2s 6ms/step - loss: 0.1983 - accuracy: 0.7960 - val_loss: 0.1887 - val_accuracy: 0.7975
Epoch 3/10
250/250 [==============================] - 1s 5ms/step - loss: 0.1832 - accuracy: 0.7960 - val_loss: 0.1778 - val_accuracy: 0.7975
Epoch 4/10
250/250 [==============================] - 1s 6ms/step - loss: 0.1751 - accuracy: 0.7960 - val_loss: 0.1718 - val_accuracy: 0.7975
Epoch 5/10
250/250 [==============================] - 2s 6ms/step - loss: 0.1705 - accuracy: 0.7960 - val_loss: 0.1682 - val_accuracy: 0.7975
Epoch 6/10
250/250 [==============================] - 1s 5ms/step - loss: 0.1677 - accuracy: 0.7960 - val_loss: 0.1660 - val_accuracy: 0.7975
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.1660 - accuracy: 0.7960 - val_loss: 0.1646 - val_accuracy: 0.7975
Epoch 

#### Predict the results using 0.5 as a threshold

In [ ]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.23629677]
 [0.23629233]
 [0.23628113]
 ...
 [0.23629001]
 [0.23629111]
 [0.23630056]]


In [ ]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


#### Print the Accuracy score and confusion matrix

In [ ]:
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

[[1595    0]
 [ 405    0]]


In [ ]:
print((cm1[0][0],cm1[1][1],cm1[0][1],cm1[1][0]))

(1595, 0, 0, 405)


In [ ]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '% of testing data was classified correctly')

79.75 % of testing data was classified correctly


#### Optimize the model

Some important parameters to look out for while optimizing neural networks are:

-Type of architecture

-Number of Layers

-Number of Neurons in a layer

-Regularization parameters

-Learning Rate

-Type of optimization / backpropagation technique to use

-Dropout rate

-Weight sharing

##### Number of Layers:
We will keep it similar to the above model so that we can compare the accuracy.
1 hidden layer.

##### Activation:
input layer: relu becasue we are in an input layer. uses the ReLu activation function for  ϕ
output layer: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for  ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

##### Type of optimization / backpropagation technique to use: 
We will use Adam. Adam is a very efficeint variation of Stochastic Gradient Descent. For Adam and its variant, learning rate or the decay rate does not really matter too much.

##### Learning Rate:
default learning rate 0.001.

##### Number of Neurons in a layer:
We will keep it 6 as per our initial calculation above.

##### Weight sharing / kernel_initializer: 
uniform the distribution with which we randomly initialize weights for the nodes in this layer.

##### Loss:
loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

Rebuilding the model using these optimised parameters

In [ ]:
# Initializing the ANN
Nclassifier = Sequential()
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
Nclassifier.add(Dense(activation = 'relu', input_dim = 11, units=48, kernel_initializer='uniform'))

In [ ]:
# Adding the hidden layer
# Notice that we do not need to specify input dim. 
Nclassifier.add(Dense(activation = 'relu', units=24, kernel_initializer='uniform')) 

In [ ]:
# Adding the hidden layer
# Notice that we do not need to specify input dim. 
Nclassifier.add(Dense(activation = 'relu', units=12, kernel_initializer='uniform'))

In [ ]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
Nclassifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [ ]:
Nclassifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
Nclassifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 48)                576       
                                                                 
 dense_5 (Dense)             (None, 24)                1176      
                                                                 
 dense_6 (Dense)             (None, 12)                300       
                                                                 
 dense_7 (Dense)             (None, 1)                 13        
                                                                 
Total params: 2,065
Trainable params: 2,065
Non-trainable params: 0
_________________________________________________________________


In [ ]:
Nclassifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=50,
          batch_size=32)

Epoch 1/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4965 - accuracy: 0.7958 - val_loss: 0.4299 - val_accuracy: 0.7975
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4258 - accuracy: 0.7960 - val_loss: 0.4230 - val_accuracy: 0.7975
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4191 - accuracy: 0.8190 - val_loss: 0.4118 - val_accuracy: 0.8440
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4115 - accuracy: 0.8324 - val_loss: 0.4054 - val_accuracy: 0.8405
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4045 - accuracy: 0.8331 - val_loss: 0.4001 - val_accuracy: 0.8415
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4001 - accuracy: 0.8369 - val_loss: 0.3992 - val_accuracy: 0.8430
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3909 - accuracy: 0.8403 - val_loss: 0.3904 - val_accuracy: 0.8410
Epoch 

#### Predict the results using 0.5 as a threshold

In [ ]:
y_pred = Nclassifier.predict(X_test)
print(y_pred)

[[0.29068533]
 [0.17198497]
 [0.08484554]
 ...
 [0.14505163]
 [0.20001805]
 [0.12767819]]


In [ ]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


#### Print the Accuracy score and confusion matrix

In [ ]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[1513   82]
 [ 210  195]]


In [ ]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '% of testing data was classified correctly')

85.4 % of testing data was classified correctly
